In [1]:
# Modules to support development
import os
import re
import collections
import itertools
import functools
import logging
import pprint
import numpy as np
import heapq
import copy
from tqdm import tqdm
import math

In [2]:
class InfiniteGrid(collections.defaultdict):
    def __init__(self, default='.'):
        super().__init__(lambda: collections.defaultdict(lambda: default))
        self.dimensions_x = None
        self.dimensions_y = None
        
    def __setitem__(self, pos, val):
        if type(pos) is int:
            return super().__setitem__(pos, val)
        
        yy, xx = pos

        if self.dimensions_x is None:
            self.dimensions_x = (xx, xx)
        else:
            self.dimensions_x = (
                min(xx, self.dimensions_x[0]),
                max(xx, self.dimensions_x[1])
            )
        
        if self.dimensions_y is None:
            self.dimensions_y = (yy, yy)
        else:
            self.dimensions_y = (
                min(yy, self.dimensions_y[0]),
                max(yy, self.dimensions_y[1])
            )    
            

        super().__getitem__(yy)[xx] = val

    def __getitem__(self, pos):
        if type(pos) is int:
            return super().__getitem__(pos)

        yy, xx = pos
        return super().__getitem__(yy)[xx]

    def iterrow(self, row):
        row_data = self[row]
        for xx in range(self.dimensions_x[0], self.dimensions_x[1]+1):
            yield row_data[xx]

    def enumeraterow(self, row):
        row_data = self[row]
        for xx in range(self.dimensions_x[0], self.dimensions_x[1]+1):
            yield (row, xx), row_data[xx]

    def __str__(self):
        if self.dimensions_x is None or self.dimensions_y is None:
            return ""

        res = []
        for yy in range(self.dimensions_y[0], self.dimensions_y[1]+1):
            res.append("%02d " % yy)
            for xx in range(self.dimensions_x[0], self.dimensions_x[1]+1):
                 res.append(str(self[yy][xx]))
            res.append("\n")

        return "".join(res)  

In [7]:
def read_input(puzzle_input):
    with open(puzzle_input) as ff:
        dd = [ ll.strip() for ll in ff.readlines() ]

    map = InfiniteGrid()
    elves = set()
    for yy, ll in enumerate(dd):
        for xx, cc in enumerate(ll):
            map[yy,xx] = cc
            if cc == '#':
                elves.add((yy,xx))

    return map, elves

map, elves = read_input(os.path.join(os.path.join("..", "dat", "day23_test.txt")))
print(map)
print(elves)

00 ....#..
01 ..###.#
02 #...#.#
03 .#...##
04 #.###..
05 ##.#.##
06 .#..#..

{(4, 0), (4, 3), (3, 1), (5, 1), (1, 6), (1, 3), (4, 2), (5, 0), (5, 6), (3, 6), (5, 3), (2, 4), (1, 2), (0, 4), (6, 1), (6, 4), (3, 5), (4, 4), (5, 5), (2, 0), (1, 4), (2, 6)}


In [35]:
def part1(puzzle_input):
    map, elves = read_input(puzzle_input)
    print("Initial State")
    print(map)

    directions = ('N', 'S', 'W', 'E')
    for ii in range(10):

        proposals = {}
        n_moves = 0
        for yy, xx in elves:
            all_clear = True
            for dy, dx in ((-1,-1),(-1,0),(-1,1),(0,-1),(0,1),(1,-1),(1,0),(1,1)):
                if map[yy-dy, xx-dx] == '#':
                    all_clear = False

            if all_clear:
                proposals.setdefault((yy, xx), []).append((yy, xx))
            else:
                for direction in directions:
                    if direction == 'N' and (map[yy-1, xx] == '.' and map[yy-1, xx-1] == '.' and map[yy-1, xx+1] == '.'):
                        proposals.setdefault((yy-1, xx), []).append((yy, xx))
                        n_moves += 1
                        break
                    elif direction == 'S' and (map[yy+1, xx] == '.' and map[yy+1, xx-1] == '.' and map[yy+1, xx+1] == '.'):
                        proposals.setdefault((yy+1, xx), []).append((yy, xx))
                        n_moves += 1
                        break
                    elif direction == 'W' and (map[yy, xx-1] == '.' and map[yy-1, xx-1] == '.' and map[yy+1, xx-1] == '.'):
                        proposals.setdefault((yy, xx-1), []).append((yy, xx))
                        n_moves += 1
                        break
                    elif direction == 'E' and (map[yy, xx+1] == '.' and map[yy-1, xx+1] == '.' and map[yy+1, xx+1] == '.'):
                        proposals.setdefault((yy, xx+1), []).append((yy, xx))
                        n_moves += 1
                        break
                else:
                    proposals.setdefault((yy, xx), []).append((yy, xx))

        elves = []
        for (p_yy, p_xx), o_pos in proposals.items():
            if len(o_pos) > 1:
                elves.extend(o_pos)
            else:
                map[o_pos[0]] = '.'
                map[p_yy, p_xx] = '#'
                elves.append((p_yy, p_xx))

        directions = directions[1],  directions[2], directions[3], directions[0],

        print("End of Round", ii+1, n_moves)
        print(map)

        if n_moves == 0:
            break

    edges = [ elves[0][0], elves[0][0], elves[0][1], elves[0][1] ]
    for (yy, xx) in elves:
        assert map[yy, xx] == '#'
        edges[0] = min(edges[0], yy) # N
        edges[1] = max(edges[1], yy) # S
        edges[2] = max(edges[2], xx) # E
        edges[3] = min(edges[3], xx) # W

    height = (edges[1] - edges[0]) + 1
    width = (edges[2] - edges[3]) + 1
    area = height * width
    empty = area - len(elves)

    return empty


ans = part1(os.path.join(os.path.join("..", "dat", "day23_test.txt")))
print(ans)
assert ans == 110

ans = part1(os.path.join(os.path.join("..", "dat", "day23.txt")))
print(ans)
#assert ans == 110


Initial State
00 ....#..
01 ..###.#
02 #...#.#
03 .#...##
04 #.###..
05 ##.#.##
06 .#..#..

End of Round 1 13
-1 .....#...
00 ...#...#.
01 .#..#.#..
02 .....#..#
03 ..#.#.##.
04 #..#.#...
05 #.#.#.##.
06 .........
07 ..#..#...

End of Round 2 11
-1 ......#....
00 ...#.....#.
01 ..#..#.#...
02 ......#...#
03 ..#..#.#...
04 #...#.#.#..
05 ...........
06 .#.#.#.##..
07 ...#..#....

End of Round 3 13
-1 ......#....
00 ....#....#.
01 .#..#...#..
02 ......#...#
03 ..#..#.#...
04 #..#.....#.
05 ......##...
06 .##.#....#.
07 ..#........
08 ......#....

End of Round 4 14
-1 ......#....
00 .....#....#
01 .#...##....
02 ..#.....#.#
03 ........#..
04 #...###..#.
05 .#......#..
06 ...##....#.
07 ...#.......
08 ......#....

End of Round 5 19
-2 ......#....
-1 ...........
00 .#..#.....#
01 ........#..
02 .....##...#
03 #.#.####...
04 ..........#
05 ...##..#...
06 .#.........
07 .........#.
08 ...#..#....

End of Round 6 8
-2 ......#....
-1 ...........
00 .#..#.....#
01 .....##.#..
02 ..........#
03 #

In [38]:
def part1(puzzle_input):
    map, elves = read_input(puzzle_input)
    print("Initial State")
    print(map)

    directions = ('N', 'S', 'W', 'E')
    ii = 0
    while True:
        proposals = {}
        n_moves = 0
        for yy, xx in elves:
            all_clear = True
            for dy, dx in ((-1,-1),(-1,0),(-1,1),(0,-1),(0,1),(1,-1),(1,0),(1,1)):
                if map[yy-dy, xx-dx] == '#':
                    all_clear = False

            if all_clear:
                proposals.setdefault((yy, xx), []).append((yy, xx))
            else:
                for direction in directions:
                    if direction == 'N' and (map[yy-1, xx] == '.' and map[yy-1, xx-1] == '.' and map[yy-1, xx+1] == '.'):
                        proposals.setdefault((yy-1, xx), []).append((yy, xx))
                        n_moves += 1
                        break
                    elif direction == 'S' and (map[yy+1, xx] == '.' and map[yy+1, xx-1] == '.' and map[yy+1, xx+1] == '.'):
                        proposals.setdefault((yy+1, xx), []).append((yy, xx))
                        n_moves += 1
                        break
                    elif direction == 'W' and (map[yy, xx-1] == '.' and map[yy-1, xx-1] == '.' and map[yy+1, xx-1] == '.'):
                        proposals.setdefault((yy, xx-1), []).append((yy, xx))
                        n_moves += 1
                        break
                    elif direction == 'E' and (map[yy, xx+1] == '.' and map[yy-1, xx+1] == '.' and map[yy+1, xx+1] == '.'):
                        proposals.setdefault((yy, xx+1), []).append((yy, xx))
                        n_moves += 1
                        break
                else:
                    proposals.setdefault((yy, xx), []).append((yy, xx))

        elves = []
        for (p_yy, p_xx), o_pos in proposals.items():
            if len(o_pos) > 1:
                elves.extend(o_pos)
            else:
                map[o_pos[0]] = '.'
                map[p_yy, p_xx] = '#'
                elves.append((p_yy, p_xx))

        directions = directions[1],  directions[2], directions[3], directions[0],

        print("End of Round", ii+1, n_moves)
        #print(map)

        if n_moves == 0:
            break

        ii += 1

    return ii + 1


ans = part1(os.path.join(os.path.join("..", "dat", "day23_test.txt")))
print(ans)
assert ans == 20

ans = part1(os.path.join(os.path.join("..", "dat", "day23.txt")))
print(ans)


Initial State
00 ....#..
01 ..###.#
02 #...#.#
03 .#...##
04 #.###..
05 ##.#.##
06 .#..#..

End of Round 1 13
End of Round 2 11
End of Round 3 13
End of Round 4 14
End of Round 5 19
End of Round 6 8
End of Round 7 10
End of Round 8 11
End of Round 9 8
End of Round 10 9
End of Round 11 7
End of Round 12 6
End of Round 13 5
End of Round 14 6
End of Round 15 6
End of Round 16 4
End of Round 17 2
End of Round 18 2
End of Round 19 2
End of Round 20 0
20
Initial State
00 ..#.#.#.....#.#.###...#..#.#.##...#..#.#....#....#.###..#.#.##.#.#####.
01 ...#......#.##.#......#...#.##.....#.#.##.#####..##.##.#...#..##..#.#..
02 ##....###.###..###....#.#..####...#####..#..###.##...########..#.####..
03 #.#..###.#.#..#...###.......##.###.##.###.##.#..##..#.#.#..#..#..#####.
04 ##.#..###.##..##..#.###.....#...#..#.##.#.###.####..#.###.#.#.###..###.
05 #.#####.####......###.###..#..#...#.......##.###...#.#.#...######.#.###
06 ..###.#.#...##.###.#.####.###.##...###.###..#...#.#######.##.#.##.#.##.
07 ##.##